## Word2Vec Example

In [ ]:
# loading the data

with open("raw_text.txt", "r", encoding="utf-8") as f:
    raw_text = f.readlines()

print("Total number of character:", len(raw_text))
print("------------------------------")
print(raw_text[:100])

In [ ]:
for i in range(len(raw_text)):
    raw_text[i] = raw_text[i].lower().replace('\n','')

print(raw_text)

In [ ]:
## Remove stop words and tokenize

stopwords = ['the', 'is','are','can','will','be', 'a','only', 'their','now','and','at','it']

filter_data = []

for sent in raw_text:
    temp_list = []
    for word in sent.split():
        if word not in stopwords:
            temp_list.append(word)
    filter_data.append(temp_list)
print(filter_data)

In [ ]:
bigrams = []
for words_list in filter_data:
    for i in range(len(words_list) - 1):
        for j in range(i+1, len(words_list)):
            bigrams.append([words_list[i], words_list[j]])
            bigrams.append([words_list[j], words_list[i]])
            

print(bigrams)

In [ ]:
all_words = []
for bi in bigrams:
    all_words.extend(bi)

all_words = list(set(all_words))
all_words.sort()

print(all_words)
print("Total number of unique words are:", len(all_words))

In [ ]:
words_dict = {}

counter = 0
for word in all_words:
    words_dict[word] = counter
    counter += 1

print(words_dict)

In [ ]:
import numpy as np

onehot_data = np.zeros((len(all_words), len(all_words)))

for i in range(len(all_words)):
    onehot_data[i][i] = 1

onehot_dict = {}
counter = 0
for word in all_words:
    onehot_dict[word] = onehot_data[counter]
    counter += 1

for word in onehot_dict:
    print(word, ":", onehot_dict[word])

In [ ]:
X = []
Y = []

for bi in bigrams:
    X.append(onehot_dict[bi[0]])
    Y.append(onehot_dict[bi[1]])
    
X = np.array(X)
Y = np.array(Y)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

embed_size = 2

model = Sequential([
    Dense(embed_size, activation='linear'),
    Dense(Y.shape[1], activation = 'softmax')
])

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')


In [ ]:
model.fit(X, Y, epochs = 1000, batch_size = 256, verbose = False)

In [ ]:
weights = model.get_weights()[0]

word_embeddings = {}
for word in all_words:
    word_embeddings[word] = weights[words_dict[word]]

print(word_embeddings)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (14, 8))
for word in list(words_dict.keys()):
    coord = word_embeddings.get(word)
    plt.scatter(coord[0], coord[1])
    plt.annotate(word, (coord[0], coord[1]))

plt.savefig('img.jpg')